# Transfer Learning - VGGNet

**Załadujemy gotowy model (w tym przypadku VGGNet19) i dostroimy go do nowego zadania klasyfikacji binarnej - przykład z książki "Uczenie głębokie i sztuczna inteligencja. Interaktywny przewodnik ilustrowany"**

### Ładujemy zależności

In [1]:
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import PIL

### Ładujemy wytrenowany model VGG19

In [2]:
vgg19 = VGG19(include_top=False, #ostatnie warstwy zagęszczone, specyficzne dla zbioru, nie zostają załadowane
              weights='imagenet',
              input_shape=(224,224,3),
              pooling=None)

In [11]:
vgg19.summary()

Model: "vgg19"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

### "Zamrażamy" warstwy modelu VGGNet19 - nie będą się douczać

In [3]:
for layer in vgg19.layers:
    layer.trainable = False

### Tworzymy całościowy model, "u góry" dodajemy VGGNet19, a dalej własne warstwy

**Zadanie 3. Dodaj warstwę spłaszczającą, dropout 0.5 i warstwę pozwalającą sklasyfikować binarnie obiekty**

In [4]:
model = Sequential()
model.add(vgg19)
model.add(Flatten())
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))


In [5]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg19 (Functional)           (None, 7, 7, 512)         20024384  
_________________________________________________________________
flatten (Flatten)            (None, 25088)             0         
_________________________________________________________________
dropout (Dropout)            (None, 25088)             0         
_________________________________________________________________
dense (Dense)                (None, 1)                 25089     
Total params: 20,049,473
Trainable params: 25,089
Non-trainable params: 20,024,384
_________________________________________________________________


### Kompilujemy model

In [6]:
model.compile(optimizer='nadam', loss='categorical_crossentropy', metrics=['accuracy'])

### Pobierz zbiór danych z [Kaggle'a](https://www.kaggle.com/datasets/dansbecker/hot-dog-not-hot-dog) i rozpakuj

### Zbiór jest mały, więc tworzymy instancje klasy ImageDataGenerator, które pozwolą nam powiększyć dane

In [7]:
train_datagen = ImageDataGenerator(
    rescale=1.0/255,
    data_format='channels_last',
    rotation_range=30,
    horizontal_flip=True,
    fill_mode='reflect')

valid_datagen = ImageDataGenerator(
    rescale=1.0/255,
    data_format='channels_last')

### Deklarujemy rozmiar paczki

In [8]:
batch_size = 32

### Powiększamy dane

In [9]:
train_generator = train_datagen.flow_from_directory(
    directory= 'archive/train',
    target_size=(224, 224),
    classes=['hot_dog','not_hot_dog'],
    class_mode='categorical',
    batch_size=batch_size,
    shuffle=True,
    seed=42)

valid_generator = valid_datagen.flow_from_directory(
    directory= 'archive/test',
    target_size=(224, 224),
    classes=['hot_dog','not_hot_dog'],
    class_mode='categorical',
    batch_size=batch_size,
    shuffle=True,
    seed=42)

Found 498 images belonging to 2 classes.
Found 500 images belonging to 2 classes.


**Zadanie 4. Wypisz co oznaczają wszystkie zadeklarowane parametry generatorów danych**

directory - ścieżka do folderu
target_size - krotka wymairów, do których znalezione obrazy będą przeskalowane
classes - opcjonalna lista klas
class_mode - determinuje typ zwracanej listy etykiek
batch_size - wielkość  partii
shuffle - przetasowanie danych
seed - ziarno generatora losowania

### Uczymy model z danymi z generatora (dla ułatwienia używamy do tego oddzielnej funkcji, choć będzie w następnej wersji tensorflowa usunięta)

In [10]:
model.fit_generator(train_generator, steps_per_epoch=15, epochs=16, validation_data=valid_generator, validation_steps=15)

C:\Users\Marcin\anaconda3\envs\tf-gpu\lib\site-packages\tensorflow\python\keras\engine\training.py:1940: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/16
15/15 [==============================] - 38s 1s/step - loss: 1.1624 - accuracy: 0.5215 - val_loss: 1.1969 - val_accuracy: 0.6021
Epoch 2/16
15/15 [==============================] - 13s 866ms/step - loss: 0.8209 - accuracy: 0.6062 - val_loss: 0.5228 - val_accuracy: 0.6469
Epoch 3/16
15/15 [==============================] - 13s 884ms/step - loss: 0.7528 - accuracy: 0.6288 - val_loss: 0.6702 - val_accuracy: 0.6521
Epoch 4/16
15/15 [==============================] - 13s 883ms/step - loss: 0.5653 - accuracy: 0.6685 - val_loss: 1.3036 - val_accuracy: 0.6062
Epoch 5/16
15/15 [==============================] - 14s 909ms/step - loss: 0.5508 - accuracy: 0.6867 - val_loss: 0.4562 - val_accuracy: 0.6906
Epoch 6/16
15/15 [==============================] - 15s 964ms/step - loss: 0.3322 - accuracy: 0.7333 - val_loss: 0.4312 - val_accuracy: 0.7146
Epoch 7/16
15/15 [==============================] - 15s 998ms/step - loss: 0.6698 - accuracy: 0.6738 - val_loss: 0.4217 - val_accuracy: 0.7240
Ep